# Exercise Chapter 9 
Industrial Statistics: A Computer Based Approach with Python<br>
by Ron Kenett, Shelemyahu Zacks, Peter Gedeck

Publisher: Springer International Publishing; 1st edition (2023) <br>
<!-- ISBN-13: 978-3031075650 -->

(c) 2022 Ron Kenett, Shelemyahu Zacks, Peter Gedeck

The code needs to be executed in sequence.

In [ ]:
import os
os.environ['OUTDATED_IGNORE'] = '1'
import warnings
from outdated import OutdatedPackageWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=OutdatedPackageWarning)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import lifelines
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
import mistat

# Exercise 6

In [ ]:
def labeledBox(ax, corner, sides, label):
    corner = list(corner)
    corner[1] = corner[1] - sides[1]/2
    ax.add_patch(patch.Rectangle(corner, *sides, facecolor='white', edgecolor="black", zorder=10))
    ax.annotate(label, (corner[0] + sides[0] / 2, corner[1] + sides[1] / 2), color='black',
                    fontsize=14, ha='center', va='center', zorder=11)

import matplotlib.patches as patch
fig, ax = plt.subplots(figsize=[6, 4])

x1 = 1; yM1 = 3; yM2 = 2
x3 = x1 + 6
ax.plot((0, x1), (0, 0), color='black')
ax.plot((x1, x1), (-yM2, yM1), color='black')
ax.plot((x3, x3), (-yM2, yM1), color='black')
ax.plot((x3, x3 + 3), (0, 0), color='black')
# M1
x2 = 3; y1 = yM1
ax.plot((x1, x3), (y1, y1), color='black')
ax.plot((x2, x2), (y1+2, y1-2), color='black')
ax.plot((x2, x2+3), (y1+2, y1+2), color='black')
ax.plot((x2, x2+3), (y1-2, y1-2), color='black')
ax.plot((x2+3, x2+3), (y1+2, y1-2), color='black')
xb = 4
labeledBox(ax, (xb, y1+2), (1, 1), r'$R_1$')
labeledBox(ax, (xb, y1), (1, 1), r'$R_2$')
labeledBox(ax, (xb, y1-2), (1, 1), r'$R_3$')
# M2
ax.plot((x1, x3), (-yM2, -yM2), color='black')
labeledBox(ax, (x1+2, -yM2), (1, 1), r'$R_4$')
labeledBox(ax, (x1+4, -yM2), (1, 1), r'$R_5$')

labeledBox(ax, (x1+7, 0), (1, 1), r'$R_6$')

corner = [2.5, -3]; sides=[4, 2]
ax.add_patch(patch.Rectangle(corner, *sides, facecolor='none', edgecolor="grey", zorder=-1))
corner = [2.5, 0]; sides=[4, 6]
ax.add_patch(patch.Rectangle(corner, *sides, facecolor='none', edgecolor="grey", zorder=-1))
ax.annotate('M1', (2,5.5), color='black',
          fontsize=14, ha='center', va='center', zorder=11)
ax.annotate('M2', (2,-1.5), color='black',
          fontsize=14, ha='center', va='center', zorder=11)

ax.set_ylim(-3.5, 6.5)
plt.axis('off')
plt.show()

# Exercise 19

In [ ]:
np.random.seed(1)
n = 100
lnTi = np.log(sorted(stats.weibull_min(2.5, scale=10).rvs(n)))
Wi = [np.log(-np.log(1 - i / (n+1))) for i in range(1, n + 1)]
df = pd.DataFrame({'Wi': Wi, 'lnTi': lnTi})
model = smf.ols('lnTi ~ Wi + 1', data=df).fit()
print(model.params)
intercept, slope = model.params
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot((Wi[0], Wi[-1]),
        (slope * Wi[0] + intercept, slope * Wi[-1] + intercept),
        color='grey')
ax.scatter(Wi, lnTi, color='black')
ax.set_xlabel(r'$W_{(i)}$')
ax.set_ylabel(r'$\ln(T_i)$')
plt.show()

In [ ]:
beta = np.exp(intercept)
nu = 1 / slope
print(f'beta {beta:.3f}, nu {nu:.3f}')

# Exercise 20

In [ ]:
Xi = [94.9, 106.9, 229.7, 275.7, 144.5, 112.8, 159.3, 153.1,
        270.6, 322.0, 216.4, 544.6, 266.2, 263.6, 138.5, 79.0,
        114.6, 66.1, 131.2, 91.1]
n = len(Xi)
Zi = [stats.norm().ppf((i - 3/8)/(n + 1/4)) for i in range(1, n+1)]

df = pd.DataFrame({'Zi': Zi, 'lnXi': np.log(sorted(Xi))})
model = smf.ols('lnXi ~ Zi + 1', data=df).fit()

intercept, slope = model.params
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot((Zi[0], Zi[-1]),
        (slope * Zi[0] + intercept, slope * Zi[-1] + intercept),
        color='grey')
ax.scatter(Zi, df['lnXi'], color='black')
ax.set_xlabel(r'$Z_{(i)}$')
ax.set_ylabel(r'$\ln(X_(i))$')
plt.show()

In [ ]:
print(model.summary2())

In [ ]:
mu = intercept
sigma = slope
print(f'mu {mu:.3f}, sigma {sigma:.3f}')

# Exercise 21

In [ ]:
np.random.seed(1)
data = [13, 157, 172, 176, 249, 303, 350, 400, 400]
n = len(data)
lnTi = np.log(data)
Wi = [np.log(-np.log(1 - i / (n+1))) for i in range(1, n + 1)]
# exclude the censored data for the regression analysis
df = pd.DataFrame({'Wi': Wi[:7], 'lnTi': lnTi[:7]})
model = smf.ols('lnTi ~ Wi + 1', data=df).fit()
print(model.params)
intercept, slope = model.params
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot((Wi[0], Wi[-1]),
        (slope * Wi[0] + intercept, slope * Wi[-1] + intercept),
        color='grey')
ax.scatter(Wi[:-2], lnTi[:-2], color='black')
ax.scatter(Wi[-2:], lnTi[-2:], color='lightgrey')
ax.set_xlabel(r'$W_{(i)}$')
ax.set_ylabel(r'$\ln(T_i)$')
plt.show()

# Exercise 22

In [ ]:
elecfail = mistat.load_data('ELECFAIL.csv')

kmf = lifelines.KaplanMeierFitter()
kmf.fit(elecfail)
kmf.plot_survival_function()

# Exercise 23

In [ ]:
elecfail = mistat.load_data('ELECFAIL.csv')

kmf = lifelines.ExponentialFitter()
kmf.fit(elecfail)
kmf.print_summary()

# Exercise 24

In [ ]:
T = [96.88, 154.24, 67.44, 191.72, 173.36, 200, 140.81, 200,
     154.71, 120.73, 24.29, 10.95, 2.36, 186.93, 57.61, 99.13,
     32.74, 200, 39.77, 39.52]
E = [ti < 200 for ti in T]

kmf = lifelines.ExponentialFitter()
kmf.fit(T, E)
kmf.print_summary()

# Exercise 26

In [ ]:
T = mistat.load_data('WEIBUL.csv')

kmf = lifelines.WeibullFitter()
kmf.fit(T)
kmf.print_summary()

In [ ]:
n_boot=5
idx = list(range(len(T)))
def stat_func(x):
    epf = lifelines.WeibullFitter().fit(T[x])
    return epf.params_['lambda_']

ci, dist = pg.compute_bootci(idx, func=stat_func, n_boot=n_boot, confidence=0.95,
                             method='per', seed=1, return_dist=True)
print(f'std(beta): {np.std(dist)}')

def stat_func(x):
    epf = lifelines.WeibullFitter().fit(T[x])
    return epf.params_['rho_']
ci, dist = pg.compute_bootci(idx, func=stat_func, n_boot=n_boot, confidence=0.95,
                             method='per', seed=1, return_dist=True)
print(f'nu(std): {np.std(dist)}')

# Exercise 31

In [ ]:
factor = 0.01*np.exp(-300/500) + 0.99*np.exp(-300/1000)
integral = 0.01*500*np.exp(-300/500) + 0.99*1000*np.exp(-300/1000)
with_burn_in = 300 + integral/factor
no_burn_in = 0.01 * 500 + 0.99 * 1000
with_burn_in - no_burn_in